# Расстояние (дивергенция) Кульбака — Лейблера:

Чтобы понять, откуда взялась Cross-Entropy, рассмотрим сначала расстояние Кульбака — Лейблера.

В математической статистике и теории информации мерой расхождения между двумя вероятностными распределениями $P$ и $Q$ является расстояние (дивергенция) Кульбака — Лейблера, вычисляемое по формуле:
$$D_{KL}(P||Q) = ∑_i P(i)\log\frac{P(i)}{Q(i)}$$

Попробуем разобраться, что значит эта формула, на примере двух монеток:
- настоящей с вероятностями выпадения орла и решки 0.5 и 0.5 соответственно,
- фальшивой с вероятностями выпадения орла и решки 0.2 и 0.8 соответственно.

Возьмем настоящую монету и произведем 10 бросков (выборок). Получили последовательность $\large \color{blue}{О О} \color{green}{Р} \color{blue}{О О} \color{green}{Р} \color{blue}{О О О} \color{green}{Р}$, где $\large \color{blue}{О}$ — это орел, $\large \color{green}{Р}$ — это решка.
Посчитаем вероятности выбросить такую последовательность для настоящей и фальшивой монеты. Броски независимые, поэтому значения вероятностей перемножаются.

<img src="https://ml.gan4x4.ru/msu/additions/L02/kl_divergence.png" width="900">

Запишем пропорцию вероятностей данной комбинации для настоящей монетки и для фальшивой (независимые случайные величины, вероятности перемножаются). Для заданных значений вероятностей пропорция будет примерно $149:1$. При пропорции $1:1$ монетки будут идентичны.

$$\frac{\color{blue}{p_1^{N_о}}\color{green}{p_2^{N_р}}}
{\color{blue}{q_1^{N_о}}\color{green}{q_2^{N_р}}}=
\frac{\color{blue}{\left(\frac{1}{2}\right)^{7}}\color{green}{\left(\frac{1}{2}\right)^{3}}}
{\color{blue}{\left(\frac{1}{5}\right)^{7}}\color{green}{\left(\frac{4}{5}\right)^{3}}}\approx \frac{149}{1}$$

Возьмем логарифм от этого значения (это позволит нам избавиться от степеней и заменить умножение сложением) и нормируем на количество бросков монетки $N=\color{blue}{N_о}+\color{green}{N_р}$.

$$\frac{\color{blue}{N_о}}{N}\log{\color{blue}{p_1}}+
\frac{\color{green}{N_р}}{N}\log{\color{green}{p_2}}-
\frac{\color{blue}{N_о}}{N}\log{\color{blue}{q_1}}-
\frac{\color{green}{N_р}}{N}\log{\color{green}{q_2}}$$

При увеличении количества бросков $N\to∞$, так как мы бросали настоящую монетку:

$$\frac{\color{blue}{N_о}}{N} \to \color{blue}{p1}, \frac{\color{green}{N_р}}{N} \to \color{green}{p2}$$

Получаем расстояние Кульбака — Лейблера:

$$D_{KL}(P||Q) = \color{blue}{p_1} \log{\color{blue}{p_1}}
+ \color{green}{p_2}\log{\color{green}{p_2}}
- \color{blue}{p_1}\log{\color{blue}{q_1}}
- \color{green}{p_2}\log{\color{green}{q_2}}$$

$$ = \color{blue}{p_1} \log{\color{blue}{\frac{p_1}{q_1}}}
+ \color{green}{p_2} \log{\color{green}{\frac{p_2}{q_2}}}$$

Посмотрим на разные фальшивые монетки:

In [ ]:
import numpy as np


def KL_dist(q1, q2, p1=0.5, p2=0.5):
    return p1 * np.log(p1 / q1) + p2 * np.log(p2 / q2)


print(f"q1 = 0.1, q2 = 0.9, Dkl = {KL_dist(0.1, 0.9):0.3f}")
print(f"q1 = 0.2, q2 = 0.8, Dkl = {KL_dist(0.2, 0.8):0.3f}")
print(f"q1 = 0.3, q2 = 0.7, Dkl = {KL_dist(0.3, 0.7):0.3f}")
print(f"q1 = 0.4, q2 = 0.6, Dkl = {KL_dist(0.4, 0.6):0.3f}")
print(f"q1 = 0.5, q2 = 0.5, Dkl = {KL_dist(0.5, 0.5)}")

q1 = 0.1, q2 = 0.9, Dkl = 0.511
q1 = 0.2, q2 = 0.8, Dkl = 0.223
q1 = 0.3, q2 = 0.7, Dkl = 0.087
q1 = 0.4, q2 = 0.6, Dkl = 0.020
q1 = 0.5, q2 = 0.5, Dkl = 0.0


Обратим внимание, что если $P=Q$, то

$$D_{KL}(P||Q) = ∑_i P(i)\log\frac{P(i)}{Q(i)} = 0$$

# Переход к оценке модели

Мы научились определять близость двух распределений. Как это поможет нам оценить качество модели, если мы знаем, какие метки классов должны получиться?

Пусть $P$ — вероятности истинных меток классов для объекта (1 для правильного класса, 0 для остальных), $Q(\theta)$ — вероятности классов для объекта, предсказанные моделью с обучаемыми параметрами $\theta$.

Расстояние Кульбака — Лейблера между истинными и предсказанными значениями:

$$D_{KL}(P||Q(\theta)) = ∑_i P(i)\log\frac{P(i)}{Q(i| \theta)} =$$

$$ = ∑_i P(i)\log{P(i))} -  ∑_i P(i)\log{Q(i|\theta)} =$$

$$ = - H(P) + H(P|Q(\theta))$$

Мы разбили сумму на две части, первая из которых называется **энтропией** $H(P)$ и не будет зависеть от модели, а вторая называется **кросс-энтропией** $H(P|Q(\theta))$.



# Энтропия и кросс-энтропия

Понятие энтропии пришло из теории связи. Для расчета энтропии можно использовать формулу Шеннона:

$$H(P)=-\sum^C_{i=1}P(i)\cdot \log_{2}(P(i)),$$
где $C$ — количество классов.


В формуле Шеннона не случайно используется логарифм по основанию 2. Она рассчитывает, **сколько бит информации минимально необходимо для передачи одного значения**, например, одного исхода броска монетки.

Если монетка **всегда выдает орел**, то нам нет смысла передавать информацию, чтобы предсказать исход:
$$H(\color{blue}{p_1 = 1}, \color{green}{p_2 = 0}) = 0$$

Для настоящей монетки необходимо будет передавать 1 бит информации на бросок:
$$H(\color{blue}{p_1 = 0.5},  \color{green}{p_2 = 0.5})= \log_{2}(2) = 1$$

А вот с поддельной монеткой получается интереснее: если сделать много бросков, можно выявить закономерность, что решка выпадет чаще (вероятность трех решек подряд будет больше вероятности одного орла), и за счет этого сократить количество передаваемой информации так, чтобы на один бросок получалось меньше 1 бита. Как это сделать — отдельная область теории информации, называемая **кодирование источника**.
$$H(\color{blue}{q_1 = 0.2}, \color{green}{q_2 = 0.8}) = 0.722$$

__Энтропия__ — мера неуверенности, связанная с распределением $P$.
Зная истинное распределение случайной величины, мы можем рассчитать его энтропию.

Если мы попытаемся использовать статистические данные, полученные для фальшивой монеты $Q$, для настоящей $P$, мы не получим выигрыш в количестве передаваемой информации:

$$H(P||Q) = - \sum^C_{i=1}P(i)\cdot \log_2(Q(i)) = 1.322$$

Формула выше называется **кросс-энтропия**.

**Кросс-энтропия** позволяет оценить ситуацию, когда мы аппроксимируем истинное распределение $P$ предсказанным распределением $Q$. Чем больше значения кросс-энтропии, тем больше расхождение между распределениями. Поэтому кросс-энтропию используют в качестве **функции потерь**.




Как мы показали выше, **кросс-энтропия** связана с **энтропией** и **расстоянием Кульбака — Лейблера**:
$$H(P||Q) = D_{KL}(P||Q) + H(P).$$

Посчитаем значения для наших монеток с помощью кода:

In [ ]:
# normal coin
p1 = 0.5
p2 = 0.5

# fake coin
q1 = 0.2
q2 = 0.8

In [ ]:
# Kullback–Leibler divergence
div_kl = p1 * np.log2(p1 / q1) + p2 * np.log2(p2 / q2)
print(f"Dkl(P||Q) = {div_kl:.3f}")

# Entropy normal coin
h_p = -p1 * np.log2(p1) - p2 * np.log2(p2)
print(f"H(P) = {h_p:.3f}")

# Entropy fake coin
h_q = -q1 * np.log2(q1) - q2 * np.log2(q2)
print(f"H(Q) = {h_q:.3f}")

# Cross-entropy
h_p_q = -p1 * np.log2(q1) - p2 * np.log2(q2)
print(f"H(P||Q) = {h_p_q:.3f}")
print(f"H(P||Q) = Dkl(P||Q) + H(P) = {h_p+div_kl:.3f}")

Dkl(P||Q) = 0.322
H(P) = 1.000
H(Q) = 0.722
H(P||Q) = 1.322
H(P||Q) = Dkl(P||Q) + H(P) = 1.322
